In [ ]:
!git clone https://github.com/rapidsai/rapidsai-csp-utils.git
!python rapidsai-csp-utils/colab/pip-install.py

In [ ]:
import cudf
import cuml

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/news-category-dataset/News_Category_Dataset_v3.json


In [2]:
##load data into dataframe
df = pd.read_json('/kaggle/input/news-category-dataset/News_Category_Dataset_v3.json',lines = True)
#df = cudf.DataFrame.from_pandas(pd_df)
df.head()

,link,headline,category,short_description,authors,date
0,https://www.huffpost.com/entry/covid-boosters-...,Over 4 Million Americans Roll Up Sleeves For O...,U.S. NEWS,Health experts said it is too early to predict...,"Carla K. Johnson, AP",2022-09-23
1,https://www.huffpost.com/entry/american-airlin...,"American Airlines Flyer Charged, Banned For Li...",U.S. NEWS,He was subdued by passengers and crew when he ...,Mary Papenfuss,2022-09-23
2,https://www.huffpost.com/entry/funniest-tweets...,23 Of The Funniest Tweets About Cats And Dogs ...,COMEDY,"""Until you have a dog you don't understand wha...",Elyse Wanshel,2022-09-23
3,https://www.huffpost.com/entry/funniest-parent...,The Funniest Tweets From Parents This Week (Se...,PARENTING,"""Accidentally put grown-up toothpaste on my to...",Caroline Bologna,2022-09-23
4,https://www.huffpost.com/entry/amy-cooper-lose...,Woman Who Called Cops On Black Bird-Watcher Lo...,U.S. NEWS,Amy Cooper accused investment firm Franklin Te...,Nina Golgowski,2022-09-22


In [3]:
##get details on classes

id2c = []
c2id = dict(df['category'].value_counts())
iter = 0
for c in df['category'].value_counts().keys():
    id2c.append(c)
    c2id[c] = iter
    iter +=1
print(c2id, id2c)


{'POLITICS': 0, 'WELLNESS': 1, 'ENTERTAINMENT': 2, 'TRAVEL': 3, 'STYLE & BEAUTY': 4, 'PARENTING': 5, 'HEALTHY LIVING': 6, 'QUEER VOICES': 7, 'FOOD & DRINK': 8, 'BUSINESS': 9, 'COMEDY': 10, 'SPORTS': 11, 'BLACK VOICES': 12, 'HOME & LIVING': 13, 'PARENTS': 14, 'THE WORLDPOST': 15, 'WEDDINGS': 16, 'WOMEN': 17, 'CRIME': 18, 'IMPACT': 19, 'DIVORCE': 20, 'WORLD NEWS': 21, 'MEDIA': 22, 'WEIRD NEWS': 23, 'GREEN': 24, 'WORLDPOST': 25, 'RELIGION': 26, 'STYLE': 27, 'SCIENCE': 28, 'TECH': 29, 'TASTE': 30, 'MONEY': 31, 'ARTS': 32, 'ENVIRONMENT': 33, 'FIFTY': 34, 'GOOD NEWS': 35, 'U.S. NEWS': 36, 'ARTS & CULTURE': 37, 'COLLEGE': 38, 'LATINO VOICES': 39, 'CULTURE & ARTS': 40, 'EDUCATION': 41} ['POLITICS', 'WELLNESS', 'ENTERTAINMENT', 'TRAVEL', 'STYLE & BEAUTY', 'PARENTING', 'HEALTHY LIVING', 'QUEER VOICES', 'FOOD & DRINK', 'BUSINESS', 'COMEDY', 'SPORTS', 'BLACK VOICES', 'HOME & LIVING', 'PARENTS', 'THE WORLDPOST', 'WEDDINGS', 'WOMEN', 'CRIME', 'IMPACT', 'DIVORCE', 'WORLD NEWS', 'MEDIA', 'WEIRD NEWS',

In [4]:
##Preprocessing
import nltk
from nltk import PorterStemmer, WordNetLemmatizer
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import wordnet

In [5]:
def toke(dataf):
    tokenizer = RegexpTokenizer(r'\w+')
    dataf['combined'] = dataf['headline'] + dataf['short_description']
    dataf['tokenized'] = dataf['combined'].apply(lambda x: tokenizer.tokenize(x))
    return dataf
def stop(dataf):
    dataf['stop_words_removed'] = dataf['tokenized'].apply(lambda text:[word for word in text if word not in nltk.corpus.stopwords.words('english')])
    return dataf
def stem(dataf):
    ps = PorterStemmer()
    dataf['stemmed'] = dataf['stop_words_removed'].apply(lambda text: [ps.stem(word) for word in text])
    return dataf
def lem(dataf):
    word_net = WordNetLemmatizer()
    dataf['lemmatized'] = dataf['stemmed'].apply(lambda text: [word_net.lemmatize(word) for word in text])
    return dataf

In [6]:
def parallelize_dataframe(df, func, n_cores=4):
    df_split = np.array_split(df, n_cores)
    pool = Pool(n_cores)
    df = pd.concat(pool.map(func, df_split))
    pool.close()
    pool.join()
    return df

In [7]:
from multiprocessing import  Pool
df = parallelize_dataframe(df,toke)
df.head()

,link,headline,category,short_description,authors,date,combined,tokenized
0,https://www.huffpost.com/entry/covid-boosters-...,Over 4 Million Americans Roll Up Sleeves For O...,U.S. NEWS,Health experts said it is too early to predict...,"Carla K. Johnson, AP",2022-09-23,Over 4 Million Americans Roll Up Sleeves For O...,"[Over, 4, Million, Americans, Roll, Up, Sleeve..."
1,https://www.huffpost.com/entry/american-airlin...,"American Airlines Flyer Charged, Banned For Li...",U.S. NEWS,He was subdued by passengers and crew when he ...,Mary Papenfuss,2022-09-23,"American Airlines Flyer Charged, Banned For Li...","[American, Airlines, Flyer, Charged, Banned, F..."
2,https://www.huffpost.com/entry/funniest-tweets...,23 Of The Funniest Tweets About Cats And Dogs ...,COMEDY,"""Until you have a dog you don't understand wha...",Elyse Wanshel,2022-09-23,23 Of The Funniest Tweets About Cats And Dogs ...,"[23, Of, The, Funniest, Tweets, About, Cats, A..."
3,https://www.huffpost.com/entry/funniest-parent...,The Funniest Tweets From Parents This Week (Se...,PARENTING,"""Accidentally put grown-up toothpaste on my to...",Caroline Bologna,2022-09-23,The Funniest Tweets From Parents This Week (Se...,"[The, Funniest, Tweets, From, Parents, This, W..."
4,https://www.huffpost.com/entry/amy-cooper-lose...,Woman Who Called Cops On Black Bird-Watcher Lo...,U.S. NEWS,Amy Cooper accused investment firm Franklin Te...,Nina Golgowski,2022-09-22,Woman Who Called Cops On Black Bird-Watcher Lo...,"[Woman, Who, Called, Cops, On, Black, Bird, Wa..."


In [8]:
df = parallelize_dataframe(df,stop)
df.head()

,link,headline,category,short_description,authors,date,combined,tokenized,stop_words_removed
0,https://www.huffpost.com/entry/covid-boosters-...,Over 4 Million Americans Roll Up Sleeves For O...,U.S. NEWS,Health experts said it is too early to predict...,"Carla K. Johnson, AP",2022-09-23,Over 4 Million Americans Roll Up Sleeves For O...,"[Over, 4, Million, Americans, Roll, Up, Sleeve...","[Over, 4, Million, Americans, Roll, Up, Sleeve..."
1,https://www.huffpost.com/entry/american-airlin...,"American Airlines Flyer Charged, Banned For Li...",U.S. NEWS,He was subdued by passengers and crew when he ...,Mary Papenfuss,2022-09-23,"American Airlines Flyer Charged, Banned For Li...","[American, Airlines, Flyer, Charged, Banned, F...","[American, Airlines, Flyer, Charged, Banned, F..."
2,https://www.huffpost.com/entry/funniest-tweets...,23 Of The Funniest Tweets About Cats And Dogs ...,COMEDY,"""Until you have a dog you don't understand wha...",Elyse Wanshel,2022-09-23,23 Of The Funniest Tweets About Cats And Dogs ...,"[23, Of, The, Funniest, Tweets, About, Cats, A...","[23, Of, The, Funniest, Tweets, About, Cats, A..."
3,https://www.huffpost.com/entry/funniest-parent...,The Funniest Tweets From Parents This Week (Se...,PARENTING,"""Accidentally put grown-up toothpaste on my to...",Caroline Bologna,2022-09-23,The Funniest Tweets From Parents This Week (Se...,"[The, Funniest, Tweets, From, Parents, This, W...","[The, Funniest, Tweets, From, Parents, This, W..."
4,https://www.huffpost.com/entry/amy-cooper-lose...,Woman Who Called Cops On Black Bird-Watcher Lo...,U.S. NEWS,Amy Cooper accused investment firm Franklin Te...,Nina Golgowski,2022-09-22,Woman Who Called Cops On Black Bird-Watcher Lo...,"[Woman, Who, Called, Cops, On, Black, Bird, Wa...","[Woman, Who, Called, Cops, On, Black, Bird, Wa..."


In [9]:
df = parallelize_dataframe(df,stem)
df.head()

,link,headline,category,short_description,authors,date,combined,tokenized,stop_words_removed,stemmed
0,https://www.huffpost.com/entry/covid-boosters-...,Over 4 Million Americans Roll Up Sleeves For O...,U.S. NEWS,Health experts said it is too early to predict...,"Carla K. Johnson, AP",2022-09-23,Over 4 Million Americans Roll Up Sleeves For O...,"[Over, 4, Million, Americans, Roll, Up, Sleeve...","[Over, 4, Million, Americans, Roll, Up, Sleeve...","[over, 4, million, american, roll, Up, sleev, ..."
1,https://www.huffpost.com/entry/american-airlin...,"American Airlines Flyer Charged, Banned For Li...",U.S. NEWS,He was subdued by passengers and crew when he ...,Mary Papenfuss,2022-09-23,"American Airlines Flyer Charged, Banned For Li...","[American, Airlines, Flyer, Charged, Banned, F...","[American, Airlines, Flyer, Charged, Banned, F...","[american, airlin, flyer, charg, ban, for, lif..."
2,https://www.huffpost.com/entry/funniest-tweets...,23 Of The Funniest Tweets About Cats And Dogs ...,COMEDY,"""Until you have a dog you don't understand wha...",Elyse Wanshel,2022-09-23,23 Of The Funniest Tweets About Cats And Dogs ...,"[23, Of, The, Funniest, Tweets, About, Cats, A...","[23, Of, The, Funniest, Tweets, About, Cats, A...","[23, Of, the, funniest, tweet, about, cat, and..."
3,https://www.huffpost.com/entry/funniest-parent...,The Funniest Tweets From Parents This Week (Se...,PARENTING,"""Accidentally put grown-up toothpaste on my to...",Caroline Bologna,2022-09-23,The Funniest Tweets From Parents This Week (Se...,"[The, Funniest, Tweets, From, Parents, This, W...","[The, Funniest, Tweets, From, Parents, This, W...","[the, funniest, tweet, from, parent, thi, week..."
4,https://www.huffpost.com/entry/amy-cooper-lose...,Woman Who Called Cops On Black Bird-Watcher Lo...,U.S. NEWS,Amy Cooper accused investment firm Franklin Te...,Nina Golgowski,2022-09-22,Woman Who Called Cops On Black Bird-Watcher Lo...,"[Woman, Who, Called, Cops, On, Black, Bird, Wa...","[Woman, Who, Called, Cops, On, Black, Bird, Wa...","[woman, who, call, cop, On, black, bird, watch..."


In [10]:
!mkdir /kaggle/working/nltk_data
nltk.download("wordnet", "/kaggle/working/nltk_data/")
nltk.data.path.append("/kaggle/working/nltk_data/")
import zipfile

z= zipfile.ZipFile('/kaggle/working/nltk_data/corpora/wordnet.zip')
z.extractall('/kaggle/working/nltk_data/corpora/')

[nltk_data] Downloading package wordnet to
[nltk_data]     /kaggle/working/nltk_data/...


In [11]:
df = parallelize_dataframe(df,lem)
df.head()

,link,headline,category,short_description,authors,date,combined,tokenized,stop_words_removed,stemmed,lemmatized
0,https://www.huffpost.com/entry/covid-boosters-...,Over 4 Million Americans Roll Up Sleeves For O...,U.S. NEWS,Health experts said it is too early to predict...,"Carla K. Johnson, AP",2022-09-23,Over 4 Million Americans Roll Up Sleeves For O...,"[Over, 4, Million, Americans, Roll, Up, Sleeve...","[Over, 4, Million, Americans, Roll, Up, Sleeve...","[over, 4, million, american, roll, Up, sleev, ...","[over, 4, million, american, roll, Up, sleev, ..."
1,https://www.huffpost.com/entry/american-airlin...,"American Airlines Flyer Charged, Banned For Li...",U.S. NEWS,He was subdued by passengers and crew when he ...,Mary Papenfuss,2022-09-23,"American Airlines Flyer Charged, Banned For Li...","[American, Airlines, Flyer, Charged, Banned, F...","[American, Airlines, Flyer, Charged, Banned, F...","[american, airlin, flyer, charg, ban, for, lif...","[american, airlin, flyer, charg, ban, for, lif..."
2,https://www.huffpost.com/entry/funniest-tweets...,23 Of The Funniest Tweets About Cats And Dogs ...,COMEDY,"""Until you have a dog you don't understand wha...",Elyse Wanshel,2022-09-23,23 Of The Funniest Tweets About Cats And Dogs ...,"[23, Of, The, Funniest, Tweets, About, Cats, A...","[23, Of, The, Funniest, Tweets, About, Cats, A...","[23, Of, the, funniest, tweet, about, cat, and...","[23, Of, the, funniest, tweet, about, cat, and..."
3,https://www.huffpost.com/entry/funniest-parent...,The Funniest Tweets From Parents This Week (Se...,PARENTING,"""Accidentally put grown-up toothpaste on my to...",Caroline Bologna,2022-09-23,The Funniest Tweets From Parents This Week (Se...,"[The, Funniest, Tweets, From, Parents, This, W...","[The, Funniest, Tweets, From, Parents, This, W...","[the, funniest, tweet, from, parent, thi, week...","[the, funniest, tweet, from, parent, thi, week..."
4,https://www.huffpost.com/entry/amy-cooper-lose...,Woman Who Called Cops On Black Bird-Watcher Lo...,U.S. NEWS,Amy Cooper accused investment firm Franklin Te...,Nina Golgowski,2022-09-22,Woman Who Called Cops On Black Bird-Watcher Lo...,"[Woman, Who, Called, Cops, On, Black, Bird, Wa...","[Woman, Who, Called, Cops, On, Black, Bird, Wa...","[woman, who, call, cop, On, black, bird, watch...","[woman, who, call, cop, On, black, bird, watch..."


In [12]:
df['input'] = df['lemmatized'].apply(lambda text: " ".join(word for word in text))
df['output'] = df['category'].apply(lambda x: c2id[x])

In [13]:
df.head()

,link,headline,category,short_description,authors,date,combined,tokenized,stop_words_removed,stemmed,lemmatized,input,output
0,https://www.huffpost.com/entry/covid-boosters-...,Over 4 Million Americans Roll Up Sleeves For O...,U.S. NEWS,Health experts said it is too early to predict...,"Carla K. Johnson, AP",2022-09-23,Over 4 Million Americans Roll Up Sleeves For O...,"[Over, 4, Million, Americans, Roll, Up, Sleeve...","[Over, 4, Million, Americans, Roll, Up, Sleeve...","[over, 4, million, american, roll, Up, sleev, ...","[over, 4, million, american, roll, Up, sleev, ...",over 4 million american roll Up sleev for omic...,36
1,https://www.huffpost.com/entry/american-airlin...,"American Airlines Flyer Charged, Banned For Li...",U.S. NEWS,He was subdued by passengers and crew when he ...,Mary Papenfuss,2022-09-23,"American Airlines Flyer Charged, Banned For Li...","[American, Airlines, Flyer, Charged, Banned, F...","[American, Airlines, Flyer, Charged, Banned, F...","[american, airlin, flyer, charg, ban, for, lif...","[american, airlin, flyer, charg, ban, for, lif...",american airlin flyer charg ban for life after...,36
2,https://www.huffpost.com/entry/funniest-tweets...,23 Of The Funniest Tweets About Cats And Dogs ...,COMEDY,"""Until you have a dog you don't understand wha...",Elyse Wanshel,2022-09-23,23 Of The Funniest Tweets About Cats And Dogs ...,"[23, Of, The, Funniest, Tweets, About, Cats, A...","[23, Of, The, Funniest, Tweets, About, Cats, A...","[23, Of, the, funniest, tweet, about, cat, and...","[23, Of, the, funniest, tweet, about, cat, and...",23 Of the funniest tweet about cat and dog thi...,10
3,https://www.huffpost.com/entry/funniest-parent...,The Funniest Tweets From Parents This Week (Se...,PARENTING,"""Accidentally put grown-up toothpaste on my to...",Caroline Bologna,2022-09-23,The Funniest Tweets From Parents This Week (Se...,"[The, Funniest, Tweets, From, Parents, This, W...","[The, Funniest, Tweets, From, Parents, This, W...","[the, funniest, tweet, from, parent, thi, week...","[the, funniest, tweet, from, parent, thi, week...",the funniest tweet from parent thi week sept 1...,5
4,https://www.huffpost.com/entry/amy-cooper-lose...,Woman Who Called Cops On Black Bird-Watcher Lo...,U.S. NEWS,Amy Cooper accused investment firm Franklin Te...,Nina Golgowski,2022-09-22,Woman Who Called Cops On Black Bird-Watcher Lo...,"[Woman, Who, Called, Cops, On, Black, Bird, Wa...","[Woman, Who, Called, Cops, On, Black, Bird, Wa...","[woman, who, call, cop, On, black, bird, watch...","[woman, who, call, cop, On, black, bird, watch...",woman who call cop On black bird watcher lose ...,36


In [14]:
df.to_json(r'\kaggle\working\processed.json')

**TRANSFER LEARNING ON DISTILBERT**

In [15]:
from transformers import pipeline
from transformers import AutoTokenizer, TFAutoModelForSequenceClassification
from transformers import DistilBertTokenizer, TFDistilBertModel, DistilBertConfig, TFAutoModelWithLMHead, TFAutoModel, AutoModel

from sklearn.model_selection import train_test_split

import tensorflow as tf
from tensorflow.keras.metrics import Accuracy, Recall, Precision, TruePositives, FalsePositives, TrueNegatives, FalseNegatives 

classifier = pipeline('sentiment-analysis')

import random
random.seed(42)

No model was supplied, defaulted to distilbert-base-uncased-finetuned-sst-2-english and revision af0f99b (https://huggingface.co/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.


In [16]:
MAX_LENGTH = 128
CLASSES = 42
## SET UP THE MODEL
save_directory = "distilbert-base-uncased"

# # detect and init the TPU
# tpu = tf.distribute.cluster_resolver.TPUClusterResolver.connect()

# # instantiate a distribution strategy
# tpu_strategy = tf.distribute.experimental.TPUStrategy(tpu)

# # instantiating the model in the strategy scope creates the model on the TPU
# with tpu_strategy.scope():

config = DistilBertConfig(dropout=0.2, attention_dropout=0.2)
config.output_hidden_states = False
transformer_model = TFAutoModel.from_pretrained(save_directory, from_pt=True, config = config)

input_ids_in = tf.keras.layers.Input(shape=(MAX_LENGTH,), name='input_token', dtype='int32')
input_masks_in = tf.keras.layers.Input(shape=(MAX_LENGTH,), name='masked_token', dtype='int32') 
# Build model that will go on top of DistilBERT
embedding_layer = transformer_model(input_ids_in, attention_mask=input_masks_in)[0]
X = tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(50, return_sequences=True, dropout=0.1, recurrent_dropout=0.1))(embedding_layer)
X = tf.keras.layers.GlobalMaxPool1D()(X)
X = tf.keras.layers.Dense(50, activation='relu')(X)
X = tf.keras.layers.Dropout(0.2)(X)
X = tf.keras.layers.Dense(CLASSES, activation='softmax')(X)
tf.keras.layers.Softmax(axis=-1)
model = tf.keras.Model(inputs=[input_ids_in, input_masks_in], outputs = X)

for layer in model.layers[:3]:
    layer.trainable = False

model.compile(optimizer="Adam", loss=tf.keras.losses.CategoricalCrossentropy(), metrics=[Accuracy(), Precision(), Recall(), TruePositives(), TrueNegatives(), FalsePositives(), FalseNegatives()],steps_per_execution=32)
print(model.summary())

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFDistilBertModel: ['vocab_projector.bias', 'vocab_transform.weight', 'vocab_transform.bias', 'vocab_projector.weight', 'vocab_layer_norm.bias', 'vocab_layer_norm.weight']
- This IS expected if you are initializing TFDistilBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFDistilBertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFDistilBertModel for predictions without further training.


Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_token (InputLayer)       [(None, 128)]        0           []                               
                                                                                                  
 masked_token (InputLayer)      [(None, 128)]        0           []                               
                                                                                                  
 tf_distil_bert_model (TFDistil  TFBaseModelOutput(l  66362880   ['input_token[0][0]',            
 BertModel)                     ast_hidden_state=(N               'masked_token[0][0]']           
                                one, 128, 768),                                                   
                                 hidden_states=None                                           

In [17]:
##### LOAD THE TEST DATA #####
X_train, X_test, y_train, y_test = train_test_split(df["input"], df["output"], test_size=0.20, random_state=42)

# Create X values
tokenizer = AutoTokenizer.from_pretrained(save_directory)
X_train = tokenizer(
     list(X_train),
     padding=True,
     truncation=True,
     return_tensors="tf",
     max_length = MAX_LENGTH
 )

X_test = tokenizer(
     list(X_test),
     padding=True,
     truncation=True,
     return_tensors="tf",
     max_length = MAX_LENGTH
 )
# Create Y values
y_train = pd.get_dummies(y_train)
y_test = pd.get_dummies(y_test)

In [18]:
#BATCH_SIZE = 16 * tpu_strategy.num_replicas_in_sync
#### TRAIN THE MODEL ####
history = model.fit([X_train["input_ids"],   X_train["attention_mask"]], 
          y_train, 
          batch_size=128, 
          epochs=4, 4
          verbose=1, 
          validation_split=0.2)

Epoch 1/4
1048/1048 [==============================] - 1460s 1s/step - loss: 2.2121 - accuracy: 0.0000e+00 - precision: 0.7091 - recall: 0.2216 - true_positives: 29717.0000 - true_negatives: 5485742.0000 - false_positives: 12194.0000 - false_negatives: 104379.0000 - val_loss: 1.7319 - val_accuracy: 0.0000e+00 - val_precision: 0.7394 - val_recall: 0.3380 - val_true_positives: 11331.0000 - val_true_negatives: 1370531.0000 - val_false_positives: 3994.0000 - val_false_negatives: 22194.0000
Epoch 2/4
1048/1048 [==============================] - 1472s 1s/step - loss: 1.8205 - accuracy: 0.0000e+00 - precision: 0.7298 - recall: 0.3182 - true_positives: 42670.0000 - true_negatives: 5482134.0000 - false_positives: 15802.0000 - false_negatives: 91426.0000 - val_loss: 1.5876 - val_accuracy: 0.0000e+00 - val_precision: 0.7609 - val_recall: 0.3759 - val_true_positives: 12602.0000 - val_true_negatives: 1370566.0000 - val_false_positives: 3959.0000 - val_false_negatives: 20923.0000
Epoch 3/4
1048/1048

In [21]:

model.save_weights('/kaggle/working/my_chp')